# Chesapeake Bay Data Exploration

By: Quinn Domanski

## What's going on with the Chesapeake Bay?

### Question:

Since 1984, sites have been set up around the bay to monitor multiple paramters that could effect the health of The Chesapeake Bay. The health of The Chesapeake Bay refers to the Bay's ability to support diverse aquatic life, through it's intricate natural systems. Some of these parameters include, Temperature ( &deg;C ), Salinity (ppt),  and pH. One important paramter is Total Nitrogen (TN) and Nitrate/Nitrite (NO23), as both of these can cause major problems to the health of the Chesapeake Bay, with an excess of these paramters causing massive algal blooms which consume oxygen creating dead zones (areas with low dissolved oxygen (DO) where marine life can't survive). We want to know:

**What causes increased levels of Total Nitrogen and Nitrate/Nitrite?**

In this notebook, we analyze parameters that could be correlated to Total Nitrogen and Nitrate/Nitrite, to identify if we can find a trend that causes these to increase. By detecting these patterns we hope to find out what could be causing high levels of both of these in the Bay. This leads us to an overarching question of:

**Can we find a correlation between any of the paramters that are recorded, including geographic features and nutrient levels?**

Dataset Used:

* https://eyesonthebay.dnr.maryland.gov/bay_cond/LongTermData.cfm

In [1]:
import pandas as pd
import os.path

### Load The Data

In [10]:
if not os.path.exists("data/all_data.csv"):
    print("does not exist")
    all_data = pd.read_csv("data/station_0_3.csv")
    for i in range(3, 93, 3):
        try:
            cur_csv = pd.read_csv(f"data/station_{i}_{i+3}.csv")
        except:
            if i == 90:
                cur_csv = pd.read_csv(f"data/station_90_91.csv")
                print("last\n")
            else:
                continue
        all_data = pd.concat([all_data, cur_csv])
    all_data.to_csv("data/all_data.csv")
else:
    print("exists")
    all_data_original = pd.read_csv("data/all_data.csv")
all_data_original.head()

exists


/tmp/ipykernel_11107/2836962627.py:17: DtypeWarning: Columns (26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  all_data_original = pd.read_csv("data/all_data.csv")


,Unnamed: 0,MonitoringStation,EventId,Cruise,Program,Project,Agency,Source,Station,SampleDate,...,Unit,Method,Lab,Problem,PrecisionPC,BiasPC,Details,Latitude,Longitude,TierLevel
0,0,CB1.0,99182.0,BAY500,TWQM,TRIB,MDDNR,MDDNR,CB1.0,3/4/2009,...,UG/L,L01,MDHMH,NaN,NaN,NaN,NaN,39.65622,-76.17504,T3
1,1,CB1.0,99182.0,BAY500,TWQM,TRIB,MDDNR,MDDNR,CB1.0,3/4/2009,...,UG/L,L01,MDHMH,NaN,NaN,NaN,NaN,39.65622,-76.17504,T3
2,2,CB1.0,99598.0,BAY502,TWQM,TRIB,MDDNR,MDDNR,CB1.0,4/6/2009,...,UG/L,L01,MDHMH,NaN,NaN,NaN,NaN,39.65622,-76.17504,T3
3,3,CB1.0,99598.0,BAY502,TWQM,TRIB,MDDNR,MDDNR,CB1.0,4/6/2009,...,UG/L,L01,MDHMH,NaN,NaN,NaN,NaN,39.65622,-76.17504,T3
4,4,CB1.0,100147.0,BAY504,TWQM,TRIB,MDDNR,MDDNR,CB1.0,5/5/2009,...,UG/L,L01,MDHMH,NaN,NaN,NaN,NaN,39.65622,-76.17504,T3


In [11]:
all_data_original.columns

Index(['Unnamed: 0', 'MonitoringStation', 'EventId', 'Cruise', 'Program',
       'Project', 'Agency', 'Source', 'Station', 'SampleDate', 'SampleTime',
       'TotalDepth', 'UpperPycnocline', 'LowerPycnocline', 'Depth', 'Layer',
       'SampleType', 'SampleReplicateType', 'Parameter', 'Qualifier',
       'MeasureValue', 'Unit', 'Method', 'Lab', 'Problem', 'PrecisionPC',
       'BiasPC', 'Details', 'Latitude', 'Longitude', 'TierLevel'],
      dtype='object')

#### Some columns are not needed and can removed, these columns include:
* Details: We don't need the the details columns as we have no Use for this columns, and it is mostly filled with NAN values.
* PrecisionPC: Is mostly NAN, hard to find a good metric for when the precision of the value is to low so the value should be thrown out.
* BiasPC: Same as PrecisionPC.
* Method: We don't really care about the method the labatory did for this project it is outside of this scope and we will not be comparing methods so to save compute and memory we will remove it.
* Lab: We don't really care about the lab that did this project it is oustide of the scope of the project to compare the different labs or measure their accuracy we are going to have a base level of trust for all data and labs and will return here if we find low accuracies later on.
* Agency, Program, Project, Cruise, EventId: None of these are within the scope of the projet and will be removed as columns from the dataset to save on compute and memory.
* TierLevel: This is for quality assurance, and while helpful for determining if it should be used in reports, for this project we are going to assume that all of the data is trustworthy and do not care if exact procedures were taken to get a T1.

In [20]:
all_data = all_data_original.drop(columns=['Details', 'PrecisionPC', 'BiasPC', 'Method', 'Lab', 'Agency', 'TierLevel', 'Program', 'Project', 'Cruise', 'EventId'])
all_data.rename(columns={'Unnamed: 0': "index"}, inplace = True)
all_data.head()

,index,MonitoringStation,Source,Station,SampleDate,SampleTime,TotalDepth,UpperPycnocline,LowerPycnocline,Depth,Layer,SampleType,SampleReplicateType,Parameter,Qualifier,MeasureValue,Unit,Problem,Latitude,Longitude
0,0,CB1.0,MDDNR,CB1.0,3/4/2009,09:26:00,NaN,NaN,NaN,0.0,S,D,FS1,CHLA,NaN,2.03,UG/L,NaN,39.65622,-76.17504
1,1,CB1.0,MDDNR,CB1.0,3/4/2009,09:26:00,NaN,NaN,NaN,0.0,S,D,FS2,CHLA,NaN,2.03,UG/L,NaN,39.65622,-76.17504
2,2,CB1.0,MDDNR,CB1.0,4/6/2009,09:55:00,NaN,NaN,NaN,0.0,S,D,FS1,CHLA,NaN,17.73,UG/L,NaN,39.65622,-76.17504
3,3,CB1.0,MDDNR,CB1.0,4/6/2009,09:55:00,NaN,NaN,NaN,0.0,S,D,FS2,CHLA,NaN,17.94,UG/L,NaN,39.65622,-76.17504
4,4,CB1.0,MDDNR,CB1.0,5/5/2009,10:32:00,NaN,NaN,NaN,0.0,S,D,FS1,CHLA,NaN,11.75,UG/L,NaN,39.65622,-76.17504


In [28]:
duplicate_mask = all_data['index'].duplicated()

# Print only the rows that are duplicates (subsequent occurrences)
print("Rows that are duplicates (subsequent occurrences):")
print(all_data['index'][duplicate_mask])

Rows that are duplicates (subsequent occurrences):
111226          0
111227          1
111228          2
111229          3
111230          4
            ...  
5067310    109613
5067311    109614
5067312    109615
5067313    109616
5067314    109617
Name: index, Length: 4688026, dtype: int64


### Explore Data Shape, Types, NAN values, etc.

In [21]:
all_data.shape

(5067315, 20)

#### Find an NAN values

In [22]:
all_data.isna().any() #Every Column has NA values

index                  False
MonitoringStation      False
Source                  True
Station                 True
SampleDate              True
SampleTime              True
TotalDepth              True
UpperPycnocline         True
LowerPycnocline         True
Depth                   True
Layer                   True
SampleType              True
SampleReplicateType     True
Parameter               True
Qualifier               True
MeasureValue            True
Unit                    True
Problem                 True
Latitude                True
Longitude               True
dtype: bool

#### Columns that should not have Null/NAN values:
* Latitude: We need to know where the data came from.
* Longitude: We need to know where the data came from.
* Unit: We need to knwo what unit it is collected in, if this is missing the data is invalidated.
* Parameter: We need to know what data this row is measuring.
* MeasureValue: Their should be an actual value that we can use, if this is missing the row is useless to us.

In [23]:
all_data_drop_na = all_data.dropna(subset=['Latitude', 'Longitude', 'Parameter', 'Unit', 'MeasureValue'])
print(f"We lost {all_data.shape[0] - all_data_drop_na.shape[0]} rows from the original dataframe. We have {all_data_drop_na.shape[0]} remaining.")

We lost 24909 rows from the original dataframe. We have 5042406 remaining.


#### Now I would like to create seperate columns for every paramter, this will make each parameter a feature that we can use to analyze the data easier. 

In [24]:
all_data_drop_na.pivot(index='index', values='MeasureValue', columns=['Parameter'])

ValueError: Index contains duplicate entries, cannot reshape